In [1]:
import sys
sys.path.append('/home/nero/AutomaticSleepScoring/Tuguldur') # change this to your local path

In [2]:
from phasic_tonic.analysis import PhasicTonic
from pipeline.DatasetLoader import DatasetLoader
from pipeline.helper import get_metadata
from pipeline.runtime_logger import logger_setup    
from pipeline.utils import get_sequences, preprocess

import numpy as np
import pandas as pd
import pynapple as nap

from tqdm.auto import tqdm
from scipy.io import loadmat

fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

logger = logger_setup()

CONFIG_DIR = "/home/nero/AutomaticSleepScoring/Tuguldur/data/dataset_loading.yaml"
OUTPUT_DIR1 = "/home/nero/AutomaticSleepScoring/Tuguldur/data/analysis_output/whole_posttrial5/"
OUTPUT_DIR2 = "/home/nero/AutomaticSleepScoring/Tuguldur/data/analysis_output/segmented_posttrial5/"

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()


In [3]:
from phasic_tonic.analysis import PhasicTonic
from phasic_tonic.core import get_start_end

# name = "Rat1_SD1_OD_4_posttrial5"
name = "Rat5_SD8_HC_0_posttrial4"
states_fname, hpc_fname, _ = mapped_datasets[name]

# Load the LFP data
lfpHPC = loadmat(hpc_fname)['HPC'].flatten()
# Load the states
hypno = loadmat(states_fname)['states'].flatten()

g = PhasicTonic(fs=2500)
g.detect(lfpHPC, hypno)
print(g.rem_intervals.tot_length())
print(g.tonic_intervals.tot_length())
print(g.phasic_intervals.tot_length())

/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")


316.0
305.0416
10.9584


In [5]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for name in mapped_datasets:
    metadata = get_metadata(name)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

In [7]:
per_trial_stats = []
per_epochs_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'epoch_id' : [],
    'state' : [],
    'duration' : []
}

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, hpc_fname, _ = mapped_datasets[name]
        logger.debug("Loading: {0}".format(name))

        if metadata["treatment"] == 0 or metadata["treatment"] == 1:
            n_down = n_down_cbd
        elif metadata["treatment"] == 2 or metadata["treatment"] == 3:
            n_down = n_down_rgs
        elif metadata["treatment"] == 4:
            n_down = n_down_os
            
        # Load the LFP data
        lfpHPC = loadmat(hpc_fname)['HPC'].flatten()

        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()

        hypno_length = len(hypno)
        recording_length = len(lfpHPC)//(targetFs*n_down)
        if abs(hypno_length - recording_length) > 3:
            print("Difference in hypnogram and recording length too high. Skipping.")
            continue
        else:
            if hypno_length < recording_length:
                lfpHPC = lfpHPC[:int(hypno_length*targetFs*n_down)]
            else:
                hypno = hypno[:int(recording_length)]
        
        # Skip if no REM epoch is detected
        if(not (np.any(hypno == 5))):
            logger.debug("No REM detected. Skipping.")
            continue
        elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
            logger.debug("No REM longer than 10s. Skipping.")
            continue
        
        # Preprocess
        lfpHPC_down = preprocess(lfpHPC, n_down, targetFs)
        # Phasic tonic analysis
        g = PhasicTonic(fs=targetFs)
        g.detect(lfpHPC_down, hypno)
        df = g.compute_stats()

        # Save the metadata and results
        for key in reversed(metadata):
            df.insert(0, column=key, value=metadata[key])
        
        per_trial_stats.append(df)

        # Save duration bouts
        for state, interval in [("phasic", g.phasic_intervals), ("tonic", g.tonic_intervals)]:
            for i, duration in enumerate(np.diff(interval, 1)):
                for condition in metadata.keys():
                    per_epochs_stats[condition].append(metadata[condition])
                per_epochs_stats['state'].append(state)
                per_epochs_stats['epoch_id'].append(i)
                per_epochs_stats['duration'].append(duration.item())

df_epochs = pd.DataFrame(per_epochs_stats)
df_trial = pd.concat(per_trial_stats, axis=0)

  0%|          | 0/539 [00:00<?, ?it/s]

/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home

Difference in hypnogram and recording length too high. Skipping.


/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home

Difference in hypnogram and recording length too high. Skipping.


...

/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:3

Difference in hypnogram and recording length too high. Skipping.


/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home

Difference in hypnogram and recording length too high. Skipping.


/home/nero/phasic_tonic/src/phasic_tonic/analysis.py:117: UserWarning: EEG is longer than hypnogram. Trimming EEG to match hypnogram length.
  warnings.warn("EEG is longer than hypnogram. Trimming EEG to match hypnogram length.")
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home/nero/anaconda3/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/home

In [8]:
df_trial

,rat_id,study_day,condition,treatment,trial_num,rem_start,rem_end,state,num_epochs,mean_duration,total_duration,percent_of_rem
0,5,8,HC,0,3,1331,1485,phasic,3,2.759333,8.278,5.375325
1,5,8,HC,0,3,1331,1485,tonic,4,36.430500,145.722,94.624675
2,5,8,HC,0,3,2428,2475,phasic,1,3.560000,3.560,7.574468
3,5,8,HC,0,3,2428,2475,tonic,2,21.720000,43.440,92.425532
0,5,8,HC,0,2,411,496,phasic,4,1.605500,6.422,7.555294
...,...,...,...,...,...,...,...,...,...,...,...,...
5,11,4,OR,4,3,928,1012,tonic,2,41.239000,82.478,98.188095
6,11,4,OR,4,3,1791,1928,phasic,6,1.671000,10.026,7.318248
7,11,4,OR,4,3,1791,1928,tonic,7,18.139143,126.974,92.681752
8,11,4,OR,4,3,1961,2026,phasic,3,2.070000,6.210,9.553846


In [9]:
df_epochs

,rat_id,study_day,condition,treatment,trial_num,epoch_id,state,duration
0,5,8,HC,0,3,0,phasic,1.910
1,5,8,HC,0,3,1,phasic,3.612
2,5,8,HC,0,3,2,phasic,2.756
3,5,8,HC,0,3,3,phasic,3.560
4,5,8,HC,0,3,0,tonic,48.982
...,...,...,...,...,...,...,...,...
8964,11,4,OR,4,3,14,tonic,3.288
8965,11,4,OR,4,3,15,tonic,21.242
8966,11,4,OR,4,3,16,tonic,22.984
8967,11,4,OR,4,3,17,tonic,4.924


In [10]:
df_trial.to_csv(OUTPUT_DIR1+"phasic_tonic_per_rem_epochs.csv", index=False)
df_epochs.to_csv(OUTPUT_DIR1+"phasic_tonic_per_durations.csv", index=False)